In [23]:
import networkx as nx
import pandas as pd
import numpy as np

def generate_timestep_observation(dag: nx.DiGraph, data: pd.DataFrame) -> pd.DataFrame:
    #inizialize the first row of the dataframe
    current_len = len(data)

    for node in nx.topological_sort(dag):
        print(data)
        if f"_t-" in str(node):
            variable = int(str(node)[0])
            timestamp = int(str(node)[-1])
            dag.nodes[node]['value'] = data.loc[len(data) - timestamp, variable] 
        else:
            parents = list(dag.predecessors(node))
            data.loc[current_len, node] = 0
            for parent in parents:
                data.loc[current_len, node] += compute_value(dag.nodes[node], dag.edges[parent, node], dag.nodes[parent]['value'])
            data.loc[current_len, node] += dag.nodes[node]['bias']
            dag.nodes[node]['value'] = data.loc[current_len, node]
    


def compute_value(node_data, edge_data, parent_value):
    weight = edge_data['weight'] 
    return parent_value * weight


In [24]:
import pickle

dag = pickle.load(open("DAG1.pkl", "rb"))
first_lines = pickle.load(open("first_lines.pickle", "rb"))

In [30]:
def testfunction(dag, first_lines):
    
    dag_dict = {}
    for node in nx.topological_sort(dag):
        if f"_t-" in str(node):
            idx = int(str(node)[0])
            position = int(str(node)[-1])
            dag_dict[node] = first_lines[idx][2 - position]
        else:
            dag_dict[node] = 0
            parents = list(dag.predecessors(node))
            for parent in parents:
                dag_dict[node] += dag_dict[parent] * dag.edges[parent, node]['weight']
            dag_dict[node] += dag.nodes[node]['bias']
    return dag_dict

In [31]:
#expectation 
testfunction(dag, first_lines)

{'2_t-2': 0.51,
 '1_t-2': 0.2,
 '0_t-2': 0.79,
 '2_t-1': 0.61,
 '1_t-1': 0.05,
 '0_t-1': 0.59,
 2: 1.2260000000000002,
 1: -0.59,
 0: -0.6904}

In [27]:
#reality
generate_timestep_observation(dag, first_lines)
first_lines.iloc[-1,:]

      0     1     2
0  0.79  0.20  0.51
1  0.59  0.05  0.61
      0     1     2
0  0.79  0.20  0.51
1  0.59  0.05  0.61
      0     1     2
0  0.79  0.20  0.51
1  0.59  0.05  0.61
      0     1     2
0  0.79  0.20  0.51
1  0.59  0.05  0.61
      0     1     2
0  0.79  0.20  0.51
1  0.59  0.05  0.61
      0     1     2
0  0.79  0.20  0.51
1  0.59  0.05  0.61
      0     1     2
0  0.79  0.20  0.51
1  0.59  0.05  0.61
      0     1      2
0  0.79  0.20  0.510
1  0.59  0.05  0.610
2   NaN   NaN  1.226
      0     1      2
0  0.79  0.20  0.510
1  0.59  0.05  0.610
2   NaN -0.59  1.226


0   -0.6904
1   -0.5900
2    1.2260
Name: 2, dtype: float64